<a href="https://colab.research.google.com/github/SuvvariJagadeeswari-dev/Natural-Language-Processing-NLP-for-Customer-Support/blob/main/customer_suppport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from scipy.sparse import hstack


In [55]:
df = pd.read_csv("/content/drive/MyDrive/customer_support_tickets.csv")


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8469 entries, 0 to 8468
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Ticket ID                     8469 non-null   int64  
 1   Customer Name                 8469 non-null   object 
 2   Customer Email                8469 non-null   object 
 3   Customer Age                  8469 non-null   int64  
 4   Customer Gender               8469 non-null   object 
 5   Product Purchased             8469 non-null   object 
 6   Date of Purchase              8469 non-null   object 
 7   Ticket Type                   8469 non-null   object 
 8   Ticket Subject                8469 non-null   object 
 9   Ticket Description            8469 non-null   object 
 10  Ticket Status                 8469 non-null   object 
 11  Resolution                    2769 non-null   object 
 12  Ticket Priority               8469 non-null   object 
 13  Tic

In [57]:
df = df.dropna(subset=['Customer Satisfaction Rating'])  # Target variable
df['First Response Time'] = df['First Response Time'].fillna(method='ffill')
df['Resolution'] = df['Resolution'].fillna("No resolution provided")
df['Time to Resolution'] = df['Time to Resolution'].fillna(method='bfill')


/tmp/ipython-input-57-3282574082.py:2: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['First Response Time'] = df['First Response Time'].fillna(method='ffill')
/tmp/ipython-input-57-3282574082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['First Response Time'] = df['First Response Time'].fillna(method='ffill')
/tmp/ipython-input-57-3282574082.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Resolution

In [58]:
df['First Response Time'] = pd.to_datetime(df['First Response Time'], errors='coerce')
df['Time to Resolution'] = pd.to_datetime(df['Time to Resolution'], errors='coerce')
df['Date of Purchase'] = pd.to_datetime(df['Date of Purchase'], errors='coerce')


In [59]:
df['Response_Hour'] = df['First Response Time'].dt.hour
df['Resolution_Hour'] = df['Time to Resolution'].dt.hour
df['Purchase_Year'] = df['Date of Purchase'].dt.year


In [61]:
#Clean Ticket Description

df['Cleaned_Description'] = df.apply(
    lambda row: row['Ticket Description'].replace("{product_purchased}", str(row['Product Purchased'])),
    axis=1
)


In [62]:
df['Cleaned_Description']

,Cleaned_Description
2,I'm facing a problem with my Dell XPS. The Del...
3,I'm having an issue with the Microsoft Office....
4,I'm having an issue with the Autodesk AutoCAD....
10,I'm having an issue with the Nintendo Switch. ...
11,I'm having an issue with the Microsoft Xbox Co...
...,...
8452,I'm having an issue with the MacBook Pro. Plea...
8453,The iPhone is unable to establish a stable int...
8455,"I've recently set up my Dyson Vacuum Cleaner, ..."
8466,I'm having an issue with the GoPro Action Came...


In [64]:
# : TF-IDF Vectorization on Cleaned Description


In [63]:
vectorizer = TfidfVectorizer(max_features=500)
X_text = vectorizer.fit_transform(df['Cleaned_Description'])


In [53]:
# One-Hot Encode Categorical Features

In [65]:
categorical_cols = ['Ticket Type', 'Ticket Subject', 'Ticket Status',
                    'Ticket Priority', 'Ticket Channel', 'Customer Gender']

df_encoded = pd.get_dummies(df, columns=categorical_cols, drop_first=True)


In [66]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2769 entries, 2 to 8467
Data columns (total 42 columns):
 #   Column                                   Non-Null Count  Dtype         
---  ------                                   --------------  -----         
 0   Ticket ID                                2769 non-null   int64         
 1   Customer Name                            2769 non-null   object        
 2   Customer Email                           2769 non-null   object        
 3   Customer Age                             2769 non-null   int64         
 4   Product Purchased                        2769 non-null   object        
 5   Date of Purchase                         2769 non-null   datetime64[ns]
 6   Ticket Description                       2769 non-null   object        
 7   Resolution                               2769 non-null   object        
 8   First Response Time                      2769 non-null   datetime64[ns]
 9   Time to Resolution                       2769 

In [53]:
#: Drop Unused / Non-Numeric Columns

In [67]:
X_structured = df_encoded.drop(columns=[
    'Ticket ID', 'Customer Name', 'Customer Email',
    'Ticket Description', 'Cleaned_Description', 'Resolution',
    'First Response Time', 'Time to Resolution', 'Date of Purchase',
    'Product Purchased'  # This caused your earlier error
])


In [68]:
scaler = StandardScaler()
X_structured_scaled = scaler.fit_transform(X_structured)


In [70]:
X_structured_scaled

array([[ 0.2413194 ,  0.00616123, -0.05314886, ...,  1.74592257,
        -0.70308855,  1.47865519],
       [-1.14091878,  0.00616123, -0.62971767, ...,  1.74592257,
        -0.70308855, -0.67629019],
       [ 1.49191586, -1.41554359, -1.63871309, ..., -0.57276309,
        -0.70308855, -0.67629019],
       ...,
       [ 0.6362446 , -1.41554359,  0.37927775, ..., -0.57276309,
        -0.70308855,  1.47865519],
       [ 0.8337072 ,  0.00616123, -0.34143326, ...,  1.74592257,
        -0.70308855, -0.67629019],
       [ 0.6362446 ,  0.00616123,  0.95584656, ..., -0.57276309,
         1.42229595, -0.67629019]])

In [69]:
# Normalize Structured Features
from scipy.sparse import hstack
X_final = hstack([X_text, X_structured_scaled])
y = df['Customer Satisfaction Rating']


In [53]:
#combine Text and Structured Features

In [71]:
from scipy.sparse import hstack
X_final = hstack([X_text, X_structured_scaled])
y = df['Customer Satisfaction Rating']


In [72]:
y

,Customer Satisfaction Rating
2,3.0
3,3.0
4,1.0
10,1.0
11,1.0
...,...
8452,3.0
8453,5.0
8455,1.0
8466,3.0


In [73]:
#Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X_final, y, test_size=0.2, random_state=42
)


In [74]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [53]:
# Train Random Forest Classifier

In [75]:
y_pred = model.predict(X_test)

print("Classification Report:\n")
print(classification_report(y_test, y_pred))
print("Accuracy:", round(accuracy_score(y_test, y_pred) * 100, 2), "%")


Classification Report:

              precision    recall  f1-score   support

         1.0       0.98      0.96      0.97       109
         2.0       0.93      0.95      0.94       117
         3.0       0.88      0.92      0.90       112
         4.0       0.97      0.88      0.92       108
         5.0       0.96      0.99      0.97       108

    accuracy                           0.94       554
   macro avg       0.94      0.94      0.94       554
weighted avg       0.94      0.94      0.94       554

Accuracy: 94.04 %


In [76]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report, accuracy_score

# Store models and names
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Naive Bayes": MultinomialNB(),
    "SVM (LinearSVC)": LinearSVC()
}

# Train and evaluate each model
for name, model in models.items():
    print(f"\nTraining: {name}")

    # MultinomialNB requires non-negative inputs → apply only on text features
    if name == "Naive Bayes":
        model.fit(X_train[:, :500], y_train)  # only TF-IDF features
        y_pred = model.predict(X_test[:, :500])
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

    print(f"Classification Report for {name}:\n")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred) * 100:.2f}%")



Training: Random Forest
Classification Report for Random Forest:

              precision    recall  f1-score   support

         1.0       0.98      0.96      0.97       109
         2.0       0.93      0.95      0.94       117
         3.0       0.88      0.92      0.90       112
         4.0       0.97      0.88      0.92       108
         5.0       0.96      0.99      0.97       108

    accuracy                           0.94       554
   macro avg       0.94      0.94      0.94       554
weighted avg       0.94      0.94      0.94       554

Accuracy: 94.04%

Training: Logistic Regression
Classification Report for Logistic Regression:

              precision    recall  f1-score   support

         1.0       1.00      1.00      1.00       109
         2.0       1.00      1.00      1.00       117
         3.0       1.00      1.00      1.00       112
         4.0       1.00      1.00      1.00       108
         5.0       1.00      1.00      1.00       108

    accuracy          